# Pytorch 教程
## 配套教材《深度学习入门——基于python的理论与实现》

### 将Pytorch 的函数封装成层

PyTorch 提供两个模块来帮助我们构建模型，一个是Sequential，一个是 Module。
Sequential 允许我们构建序列化的模块(网络是线性排列)，而 Module 是一种更加灵活的模型定义方式（可以实现较为复杂的网络结构）。

In [ ]:
import torch
import torch.nn as nn
# N是批大小；D是输入维度
# H是隐藏层维度；D_out是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10
#创建输入和输出随机张量
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)
# 使用nn包将我们的模型定义为一系列的层。

# nn.Sequential是包含其他模块的模块，并按顺序应用这些模块来产生其输出。

# 每个线性模块使用线性函数从输入计算输出，并保存其内部的权重和偏差张量。
# 在构造模型之后，我们使用.to()方法将其移动到所需的设备。
        
# nn包还包含常用的损失函数的定义；
# 在这种情况下，我们将使用平均平方误差(MSE)作为我们的损失函数。
# 设置reduction='sum'，表示我们计算的是平方误差的“和”，而不是平均值;
# 这是为了与前面我们手工计算损失的例子保持一致，
# 但是在实践中，通过设置reduction='elementwise_mean'来使用均方误差作为损失更为常见。
loss_fn = torch.nn.MSELoss(reduction='sum')
learning_rate = 1e-4
for t in range(500):
    # 前向传播：通过向模型传入x计算预测的y。
    # 模块对象重载了__call__运算符，所以可以像函数那样调用它们。
    # 这么做相当于向模块传入了一个张量，然后它返回了一个输出张量。
    y_pred = model(x)
     # 计算并打印损失。
     # 传递包含y的预测值和真实值的张量，损失函数返回包含损失的张量。
    loss = loss_fn(y_pred, y)
    print(t, loss.item())
    # 反向传播之前清零梯度
    model.zero_grad()
    # 反向传播：计算模型的损失对所有可学习参数的导数（梯度）。
    # 在内部，每个模块的参数存储在requires_grad=True的张量中，
    # 因此这个调用将计算模型中所有可学习参数的梯度。
    loss.backward()
    # 使用梯度下降更新权重。
    # 每个参数都是张量，所以我们可以像我们以前那样可以得到它的数值和梯度
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

In [ ]:
import torch
# N是批大小；D是输入维度
# H是隐藏层维度；D_out是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10
# 产生随机输入和输出张量
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)
# 使用nn包定义模型和损失函数
model = torch.nn.Sequential(
          torch.nn.Linear(D_in, H),
          torch.nn.ReLU(),
          torch.nn.Linear(H, D_out),
        )
loss_fn = torch.nn.MSELoss(reduction='mean')
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for t in range(500):
    # 前向传播：通过像模型输入x计算预测的y
    y_pred = model(x)
    # 计算并打印loss
    loss = loss_fn(y_pred, y)
    print(t, loss.item())
    
    # 在反向传播之前，使用optimizer将它要更新的所有张量的梯度清零(这些张量是模型可学习的权重)
    optimizer.zero_grad()
    # 反向传播：根据模型的参数计算loss的梯度
    loss.backward()
    # 调用Optimizer的step函数使它所有参数更新
    optimizer.step()

In [30]:
import torch
import torch.nn as nn
import random
N,D_in,H,D_out = 64,1000,100,10
x = torch.randn(N,D_in)
y = torch.randn(N,D_out)

class model(nn.Module):
    def __init__(self):
        super(model,self).__init__()
        self.input_linear = nn.Linear(D_in,H)
        self.middle_linear = nn.Linear(H,H)
        self.output_linear = nn.Linear(H,D_out)
    def forward(self,x):
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred

model = model()
criterion = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(5):
    # 前向传播：通过向模型传入x计算预测的y。
    y_pred = model(x)
    # 计算并打印损失
    loss = criterion(y_pred, y)
    print(t, loss.item())
    # 清零梯度，反向传播，更新权重 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 1.050244927406311
1 1.1085119247436523
2 1.0467908382415771
3 1.0467795133590698
4 1.0474863052368164
